In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import pandas as pd
from scipy.stats import t
from matplotlib.colors import ListedColormap
from scipy import stats

: 

# Exercise: Linear Regression 

We start by building a dataset (X,Y) with 50 samples, using the linear function f and add noise using np.random.normal(0,sigma):

In [ ]:
def f(x,theta):
    x_temp = np.array([1.0,x])
    return np.dot(x_temp,theta)

: 

In [ ]:
sigma = 25
n =7
x = np.linspace(0,100,n)

#our linear model is defined by beta:
beta = [3.0,1.5]

#a more compact way of coding this: 
y = np.array([ f(x[kk],beta) + np.random.normal(0,sigma) for kk in range(len(x))])
#test with random data:
#y = np.array([ np.random.random() for kk in range(len(x))])  

: 

In [ ]:
plt.title('My dataset')
plt.scatter(x,y)
plt.xlabel('x')
plt.ylabel('y')
#plt.savefig('ligreg1.png')
plt.show()

: 

<b> (a) Estimate the $\beta_0$ and $\beta_1$ from this dataset using MSE minimination (formula on slide 20):

python hints: 

to build the matrix X for x, you might have to "force" it to be a np.array: to convert a list xx to a np.array, use xx = np.array(xx). 

to invert a matrix, use np.linalg.inv. 

for matricial product, use np.dot(,)

In [ ]:
#Building the X matrix
X=np.ones((n,2))
for i in range(n):
    X[i][1]=x[i]
print(X)

: 

In [ ]:
#hatbeta1 is the slope and hatbeta0 is the intercept

from numpy.linalg import inv
from numpy.linalg import det

Xt=np.transpose(X)
XtX=np.dot(Xt,X)

inXtX=inv(XtX)
Y=np.array(y)
b=np.dot(inXtX,Xt)
hatbeta=np.dot(b,Y)
hatbeta0=hatbeta[0]
hatbeta1=hatbeta[1]
print(hatbeta)

: 

In [ ]:
print('the estimation of the slope is ',round(hatbeta[1],2), 'the estimation of the intercept is ', round(hatbeta[0],2))

: 

In [ ]:
#Gegeven output: the estimation of the slope is  1.7361474847139053 the estimation of the intercept is  -21.488178897684556
plt.figure(figsize=(5,5))

def estfunc(x,beta):
    return beta[0]+beta[1]*x
plt.scatter(x,y,label="dataset")
plt.plot(x,estfunc(x,hatbeta),label="estamination",color="orange")
plt.title("Comparison between dataset and estaminations",fontsize=15)
plt.xlabel("x",fontsize=15)
plt.ylabel("y",fontsize=15)
plt.legend()
plt.show()

: 

<b> (b) Estimate sigma from the dataset:

In [ ]:
y_hat = estfunc(x,hatbeta)

: 

In [ ]:
Xb=np.dot(X,hatbeta).reshape(n,1)
Y=Y.reshape(n,1)
YXb=Y-Xb
YXbt=np.transpose(YXb)
MSE=(1/n)*(np.dot(YXbt,YXb))
print(MSE)

: 

In [ ]:
sigma_hat =np.sqrt(n*MSE/(n-2))[0][0]
print('The estimated value of sigma is ', round(sigma_hat,2),', the true value is ', sigma)

: 

In [ ]:
x_=np.sum(x)/n
sigmabeta1 = np.sqrt(sigma**2/(np.sum((x-x_)**2)))
print('The error made on estimating the slope ',round(sigmabeta1,2),' (to be compared with the value of slope', round(hatbeta[1],2),' )')

: 

<b> (c) Compute the p value associate with this linear regression and explain what it represents. 
=> you need first to compute $z$, and then compute what is the probability of having observed a $z$ with that value of higher (in absolute value). 

Then compute $z_1$ as on slide 28 to get the boundary value of $z$ for acceptance or rejection of the null hypothesis. </b>


hint: you need to estimate the value of sigma from the dataset for this, and rely on formula for $z$ on slide 28

hint 2: use the following functions

t.pdf(...,df) = Probability Density Function (PDF): Returns the probability for an observation having a specific value from the distribution (df = number of degrees of freedom of the t distribution).

t.cdf(...,df) = Cumulative Density Function (CDF): Returns the probability for an observation equal to or lesser than a specific value from the distribution.

t.ppf(..., df) = Percent Point Function (PPF): Returns the observation value for the provided probability that is less than or equal to the provided probability from the distribution.

: 

In [ ]:
myz = (hatbeta1-0)/(sigmabeta1)
print(myz)

: 

In [ ]:
df=n-2

: 

In [ ]:
#this is the p-value: 
p = 1-t.cdf(myz, df)+t.pdf(myz, df)
print(p)

: 

In [ ]:
# here is how to do this with a black box: 
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
print(p_value)

: 

If it is less than 5 percent, I reject my null hypothesis and I claim that there is a linear relationship between $x$ and $y$. 
The quality of the fit is given by (see slide 29):

In [ ]:
y_=np.mean(y)
TSS = np.sum((y-y_)**2)
RSS = np.sum((y-hatbeta0-hatbeta1*x)**2)
1-RSS/TSS  #this is the proportion of the variance explained

: 

In [ ]:
p_max = 0.975
# retrieve value <= probability
max_z = t.ppf(p_max,df)
p_min = 0.025
# retrieve value <= probability
min_z = t.ppf(p_min,df)
print(min_z,max_z)

: 

In [ ]:

fig, ax = plt.subplots(1, 1)
df = n-2# number of degree of freedom (the -2 is because we estimated two parameters)

x_fig = np.linspace(t.ppf(0.005, df),t.ppf(0.995, df), 100)

ax.plot(x_fig, t.pdf(x_fig, df),'r-', lw=2, alpha=0.6, label='t-distribution')
ax.plot(x_fig, stats.norm.pdf(x_fig),'g--', lw=2, alpha=0.6, label='norm')


ax.vlines(myz,ymin=0,ymax=.5 )
ax.vlines(max_z,colors='r',ymin=0,ymax=.5)
ax.vlines(min_z,colors='r',ymin=0,ymax=.5)

ax.legend(loc='upper left', frameon=False)
ax.set_xlabel('z')
ax.set_ylabel('p(z)')
plt.savefig('linreg2.png')
plt.show()

: 

If the blue line is far from the peak of the distribution (futher than the red lines), it is very unlikely to observe that "z-value" under the null hypothesis => we reject it and accept the fact that there is a linear relation between x and y. 

In [ ]:
# FYI - with a black box: 
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
print(p_value)

: 

(d) it is just a check: the $z_{min}$ and $z_{max}$ values (cf slide 28) by integrating the probability distribution => it should give 0.95: 

In [ ]:
import scipy.integrate as integrate

from scipy.integrate import quad

def integrand(x, df):

    return t.pdf(x, df)

I = quad(integrand, min_z, max_z, args=(df))

I

: 

(e) Repeat (a) 100 times for different dataset (to be build similarly as above) and represent the obtained $\hat \beta_1$ with an histogram 

In [ ]:
def estbeta(x,y):
    Y=np.array(y)
    b=np.dot(inXtX,Xt)
    return np.dot(b,Y)

: 

In [ ]:
def estsigma(x,y,hatbeta):
    Xb=np.dot(X,hatbeta).reshape(n,1)
    Y=np.array(y)
    Y=Y.reshape(n,1)
    YXb=Y-Xb
    YXbt=np.transpose(YXb)
    MSE=(1/n)*(np.dot(YXbt,YXb))
    return np.sqrt(n*MSE/(n-2))[0][0]

: 

In [ ]:
number_of_iterations = 100
beta1_evaluations = np.zeros(number_of_iterations)
beta1_evaluations_over_sigma = np.zeros(number_of_iterations)
sigma_evaluations = np.zeros(number_of_iterations)

for kk in range(number_of_iterations):
    y = np.array([ f(x[ii],beta) + np.random.normal(0,sigma) for ii in range(len(x))])

    betas=estbeta(x,y)
    beta1_evaluations[kk] = betas[1]
    sigma_evaluations[kk] = estsigma(x,y,betas)
    beta1_evaluations_over_sigma[kk] = betas[1]/sigma_evaluations[kk]

: 

In [ ]:
plt.hist(beta1_evaluations)
plt.title(r'Distribution of estimations of $\hat{\beta}_1$')
plt.xlabel(r' $\hat{\beta}_1$')
plt.ylabel('freq')
plt.show()

: 

In [ ]:
if False:
    #to get info about a function, place a ? in front and run in a cell
    ?stats.chi2.fit

: 

Check that the number of dof of the chi2 distribution is given by n-2 (here 2 because we estimated two parameters):

In [ ]:
df_estimate,loc_estimate, scale_estimate  = stats.chi2.fit(sigma_evaluations)
df_estimate,loc_estimate, scale_estimate

: 

In [ ]:
#this should be a chi-2 distribution with n-p dof (so here n=7 and p=2 => 5dof)
plt.figure(tight_layout = True)

fkfq , bins_S, abc = plt.hist(sigma_evaluations, density=1, alpha=0.5)

plt.title('Distribution of estimations of $\hat{\sigma}$')
best_fit_line_chi2 = stats.chi2.pdf(bins_S,df_estimate, loc_estimate, scale_estimate)
plt.plot(bins_S,best_fit_line_chi2,label='chi2-distr')
plt.xlabel('$\hat{\sigma}$')
plt.ylabel('freq.')
plt.legend()
plt.savefig('linreg3.png')
plt.show()

: 

(f) What is the probability distriubtion of the estimated values $\hat \beta_1 / \sigma_{\beta_1}$. Comment your answer. Hint: use stats.norm.fit

In [ ]:
mu_normal_fit, sigma_normal_fit = stats.norm.fit(beta1_evaluations_over_sigma)
print(mu_normal_fit, sigma_normal_fit)

: 

In [ ]:
z_evaluations = beta1_evaluations_over_sigma

: 

In [ ]:
plt.figure(tight_layout = True)

_, bins, _ = plt.hist(z_evaluations, density=1, alpha=0.5)

best_fit_line = stats.norm.pdf(bins, mu_normal_fit, sigma_normal_fit)

df_est, loc_est, scale_est = t.fit(z_evaluations) #fiting to a t-student distribution 
best_fit_line_t = stats.t.pdf(bins,df_est, loc_est, scale_est)

plt.plot(bins, best_fit_line,label='normal')
plt.plot(bins,best_fit_line_t,label='t-distr')

plt.title('z-values')
plt.xlabel('estimation of z')
plt.ylabel('freq.')
plt.legend()
plt.show()

: 

# Exercise: Linear discriminant analysis: pink vs green

We start with a labelled data set whose inputs are : 

In [ ]:
X = np.array([1.75860969,  1.21822739, -1.26356318,  2.91537812,  3.10517506,
        3.08995169,  1.73663264,  1.417986  ,  2.97874194,  1.02612401,
        1.44476555,  0.57531973,  0.26809148,  4.09974131,  2.50400378,
        2.99089042,  0.57560659,  1.82877472,  1.4679634 ,  2.43664457,
        1.97868673, -0.80911347,  1.55322051,  1.24100934,  2.39248028,
        3.83483408,  3.1876462 ,  3.55671585,  1.38558747,  3.30074664,
        2.75590294,  2.32096285,  2.85548297,  2.32616711, -1.57455353,
        2.4311101 ,  2.55119646,  2.90450118,  2.00737857,  0.01656822,-1.19331961,  0.68029264, -1.40977742, -1.57572063, -2.65345905,
       -2.16083126,  0.8167179 ,  0.05211783, -3.24852015, -2.09097824,
       -1.77366778, -1.38219704, -3.38111373, -5.17816507, -0.97002885,
       -2.03895265, -0.99034122,  0.39721913, -0.08746797, -2.97716289,
       -4.69607405, -3.19356515, -0.90374167, -2.6196549 ,  0.2500216 ,
       -3.35646837, -1.59124805,  0.04690309, -4.48707835, -2.65187787,
       -1.30699669, -3.7194796 , -2.50237985, -1.19258543, -1.77802654,
       -2.4663718 , -0.64556797, -1.3099631 , -3.26836737,  1.89202167])

: 

In [ ]:
X.shape

: 

and outputs (here all pink outputs are listed first) :

In [ ]:
Y = np.array(['pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','pink','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green','green'])

: 

In [ ]:
Y.shape

: 

Define an array containing only the 'pink' inputs and one containing only the 'green'inputs:

In [ ]:
pink_indices = np.where(Y=='pink')[0]
pink = X[pink_indices]
green_indices = np.where(Y=='green')[0]
green = X[green_indices]

: 

## Coding a LDA 

Our goal will be to perform a linear discriminant analysis. In the case of two categories (as here with the pink and green categories), this amounts to find the boundary of decision $x^\star$. If the input $x$ is larger than $x^\star$ it belongs to the category for which the mean value of $x$ is larger, and if $x$ is smaller than $x^\star$ it belongs to the other category. 

REM: we can also decide to focuss on only one category, let say the pink category, and decide to assign a sample to this category if if the probability of belonging to this category is higher than 60% for instance (instead of the more "natural" choice of 50 %) => this amounts to change the threshold. 

Here we want to evaluate 

$p_{\text{green}} (x)$ and 

$p_{\text{pink}} (x)$.


We can use the formula: 

$p_k(x) = \frac{ \frac{\pi_k}{\sqrt{2 \pi }\sigma} \exp{(- \frac{1}{2 \sigma^2}(x - \mu_k)^2 )} }{N} $   (with $k =$ pink, green)

if we have estimations for $\pi_k$, $\mu_k$ and $\sigma$. $N$ is a normalisation factor. 



<b> Estimate of the means for both categories: 

In [ ]:
mu_p_hat=np.mean(pink)
mu_g_hat=np.mean(green)
print('Mean value of pink samples is ',round(mu_p_hat,2),'and of green samples ', round(mu_g_hat,2))


: 

Clearly, samples with a value close to 2 will be more likely to be pink and samples with a value close to -1.5 will be more likely to be green. 

<b> Estimate sigma for the whole data set (see slides):

$\hat \sigma^2  = \frac{1}{n-K} \sum_k \sum_i (x_i - \hat \mu_k)^2$ 

where $K$ is the number of categories (here 2), the first sum is over catergories and the second over the samples within the given category: 

$\hat \sigma^2  = \frac{1}{n-2} ( \sum_{i \text{ for pink samples}} (x_i - \hat \mu_p)^2 \, + \, \sum_{i \text{ for green samples}} (x_i - \hat \mu_g)^2)$ 


In [ ]:
pink_normalized = pink-mu_p_hat
green_normalized = green-mu_g_hat

sigma_hat_2 = (1/78)*((np.sum((pink_normalized**2)))+(np.sum((green_normalized**2))))
sigma_hat_2

: 

<b> Estimate $\pi_{\text{green}}$ and $\pi_{\text{pink}}$ (this is easy, it is simply the fractions of green and pink samples): 

In [ ]:
pi_green = len(green)/80
pi_pink = len(pink)/80

: 

In [ ]:
print(pi_pink,pi_green)

: 

<b> Find the boundary value $x^\star$ for decision. You need to solve (by hand) the equation $\delta_{\text{pink}}(x^\star) = \delta_{\text{green}}(x^\star)$ (see slide 19):  


$\delta_k (x) =  x \, \frac{\mu_k}{\sigma^2} - \frac{\mu_k^2}{2 \, \sigma^2} + \log{(\pi_k)} $


In the present case, we want to have $\pi_{\text{green}} = \pi_{\text{pink}}$ and the equation $\delta_{\text{pink}}(x^\star) = \delta_{\text{green}}(x^\star)$ reads

$ x^\star \, \frac{\mu_{\text{pink}}}{\sigma^2} - \frac{\mu_{\text{pink}}^2}{2 \, \sigma^2} + \log{(\pi_{\text{pink}})} = x^\star \, \frac{\mu_{\text{green}}}{\sigma^2} - \frac{\mu_{\text{green}}^2}{2 \, \sigma^2} + \log{(\pi_{\text{green}})} $

and gets actually very simple, 
 
 $ x^\star \, \mu_{\text{pink}} - \frac{\mu_{\text{pink}}^2}{2 }  = x^\star \, \mu_{\text{green}} - \frac{\mu_{\text{green}}^2}{2}  $.
 
 We deduce that
 
 $x^\star = \frac{\mu_{\text{pink}}^2 - \mu_{\text{green}}^2}{ 2 \, (\mu_{\text{pink}}-\mu_{\text{green}})}$


In [ ]:
#writing delta_pink = delta_green gives us the boundary value for an new observation 
#to be classified as green or pink 

x_boundary = (mu_p_hat**2-mu_g_hat**2)/(2*(mu_p_hat-mu_g_hat))
x_boundary

: 

<b> Plot the histogram (just need to have defined x_boundary and the plotting will work):

In [ ]:
rng = np.random.RandomState(10)  # deterministic random data
bins=np.linspace(-20,20,100)
bins=np.append(bins,np.max([np.max(pink),np.max(green)]))
a = np.hstack((pink,green))
plt.figure(figsize=(10,6))
plt.ylim(top=np.max(bins))
y1, x1,_ = plt.hist(pink, bins='auto',alpha=.4,color='pink',label='pink histogram')  # alpha level => transparent color (in case histog. overlap)
y2, x2,_ = plt.hist(green,bins='auto',alpha=.4,color='green',label='green histogram')
plt.ylim([0,1.1*np.max(np.array([y1,y2]))])
plt.axvline(x_boundary,label='boundary value')
plt.title("Histogram")
plt.xlabel('x')
plt.ylabel('frequency')
plt.legend(loc='upper left')
plt.show()

: 

If you have a new input $x$ for which we do not know the catergory, the LDA will assign this new input to the pink categroy if $x$ is bigger than $x^\star$ and to the green one if $x$ is smaller than $x^\star$ (for a threshold of 50%). 

<b> For information: the dataset has been generated with the following code</b>

In [ ]:
if False:  #I put this "if False" not to generate a new dataset
    mu_p = 2
    mu_g = -2
    sigma = 1.5
    number_points = 40
    pink = np.random.normal(mu_p,sigma,number_points)
    green = np.random.normal(mu_g,sigma,number_points)
    n = 2*number_points

: 

The actual "input" data X is given by the pink and green arrays. Here we have drawn inputs for the two categries (pink and green) from gaussian distributions with the same variance but different means. This actually corresponds to the hypothesis we make in the LDA, i.e. we assume that inside each category, samples are gaussianly distributed. We could define it as: 

In [ ]:
if False:
    X=np.concatenate((green,pink),axis=0)

: 

The "output" vector y is then y = ('green', 'green', ... , 'pink','pink') with 40 x 'green' and 40 x 'pink'. We could also assign an integer to each category. Let's say that 'green' = 0 and 'pink' = 1 for instance. Then y = (0,0,...,1,1,...). 


## Assessing the training error 

Using the threshold value of x we computed, we can see to which category all samples would be assigned. 

In [ ]:
# all 'green' elements in our labelled data set have the following "x" values: 
green

: 

Let us look which of those would been correctly assigned with our obtained threshold: (use green < x_boundary and np.sum), this is the true positive rate:

In [ ]:
#gegeven output: 35
sg=0
for i in range(40):
    if green[i]<x_boundary:
        sg+=1
sg

: 


Similarly, we will have the following number of correclty classified pink samples:

In [ ]:
#gegeven output: 36
sp=0
correct_pink=[]
for i in range(40):
    if pink[i]>x_boundary:
        sp+=1
        correct_pink.append(pink[i])
sp


: 

Let us focus on the pink category. We can compute the 

- true positive rate : number of correclty classified pink samples. 
- false positive rate : number of samples classified as pink while they are green. 



In [ ]:
#gegeven output: true positive rate is  0.9 , false positive rate is  0.125
true_positive_rate = sp/len(pink)
print('true positive rate is ',true_positive_rate)

false_positive_rate = (len(pink)-sg)/len(pink)
print('false positive rate is ',false_positive_rate)

: 


<b> Do a ROC figure.</b>

Let us focus on the pink category. The typical choice is to assign an input $x$ to the pink category is if p_pink(x)>.5. 

However we could choose to be more or less 'cautious' and decide to assign $x$ to the pink category if p_pink(x) > .1,.2,.3,.4,.5,.6,..

For this, we need to evaluate 

$$p_k(x) = \frac{ \frac{\pi_k}{\sqrt{2 \pi} \sigma} \exp{(- \frac{1}{2 \sigma^2}(x - \mu_k)^2 )} }{N} $$

We can first compute the "not normalize $p_k(x)$": 


In [ ]:
sigma_hat = np.sqrt(sigma_hat_2)


: 

In [ ]:
p_pink_notnorm = ((pi_pink/(np.sqrt(2*np.pi)*sigma_hat))*np.exp((-1/(2*(sigma_hat**2)))*((X-mu_p_hat)**2)))
p_green_notnorm =((pi_green/(np.sqrt(2*np.pi)*sigma_hat))*np.exp((-1/(2*(sigma_hat**2)))*((X-mu_g_hat)**2)))


: 

In [ ]:
mu_g_hat

: 

Compute the normalisation factor $N$ for each input:

In [ ]:
N_hat = [p_pink_notnorm[kk]+p_green_notnorm[kk] for kk in range(len(X))]

: 

Compute the p_pink and p_green

In [ ]:
p_pink = [p_pink_notnorm[kk]/N_hat[kk] for kk in range(len(X))]
p_green = [p_green_notnorm[kk]/N_hat[kk] for kk in range(len(X))]


: 

In [ ]:
#check normalisation
#[p_pink[k]+p_green[k] for k in range(len(X))]


: 

Define an array containing the probability of being pink for true pink samples, and one with the probability of pink for not pink samples : 

In [ ]:
#p_pink_for_true_pink should be an array whose length is the number of “true pink” samples 
#and which contains the probability of being pink for each of those samples. 
#You can use np.where in the output vector to find the location of the pink samples. 
#(most values should be close to one because the classes are well separated).

number_of_pink = len(pink)
p_pink_for_true_pink = []
p_pink_for_not_pink = []

for i in range(len(X)):

    if Y[i]=='pink':
        p_pink_for_true_pink.append(p_pink[i])
    elif Y[i]=='green':
        p_pink_for_not_pink.append(p_pink[i])



print(len(p_pink_for_true_pink))
print(len(p_pink_for_not_pink))

: 

In [ ]:
list_threshold = np.array([0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1])

#we define arrays filled with zeros where we will store our results
tpr = np.zeros(len(list_threshold))
fpr = np.zeros(len(list_threshold))

for k in range(len(list_threshold)):

    #compute threshold value
    #compute TPR and FPR and store value
    threshold = list_threshold[k]
    
    tpr[k]=np.sum(p_pink_for_true_pink>=threshold)/number_of_pink
    fpr[k]=np.sum(p_pink_for_not_pink>=threshold)/number_of_pink



print(np.array([list_threshold,tpr,fpr]))


: 

In [ ]:

plt.title('ROC curve for the pink category')
plt.scatter(fpr,tpr)
plt.ylabel('true positive rate')
plt.xlabel('false positive rate')
plt.xlim((-0.05,1.05))
plt.ylim((-0.05,1.05))
plt.show()

: 

Let make the graph a bit nicer by labelling each dot:

In [ ]:
colours  = ListedColormap(['r','b','yellow','cyan','pink','purple','black','grey','green','magenta','orange'])
classes = [str(list_threshold[k]) for k in range(len(list_threshold))]
values = list_threshold


: 

In [ ]:
plt.figure(figsize=(10,6))
plt.title('ROC curve for the pink category')
scatter = plt.scatter(fpr,tpr,c=values,cmap=colours)
plt.legend(handles=scatter.legend_elements()[0], labels=classes,title='threshold')
plt.ylabel('true positive rate')
plt.xlabel('false positive rate')
plt.xlim((-0.05,1.05))
plt.ylim((-0.05,1.05))
plt.plot()

: 

Comment this ROC curve: 

The lowest threshold is related to the highest false positive rate and highest true positive rate. When we take the value of the threshold more up, the false positive rate gets lower. This is a good thing. On the other hand, the true positive rate decreases also a little and this is something we do not like to see. The ideal case would be if the true positive rate was close to 1 and the false positive rate close to 0. But all the values are in the left upper corner so it is close to the ideal case. If we would increase the threshold value more and more, my prediction is that the true positive rate and the false positive rate would further decrease.

# Exercise: The simplest neural network ever

In [ ]:
# sigmoid function - this is our squeezing funct (when deriv=False) and its derivative (when deriv = True)

def nonlin(x, deriv=False):
    if (deriv == True):
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))


# input dataset
X = np.array([[0, 0, 1],
              [0, 1, 1],
              [1, 0, 1],
              [1, 1, 1]])
print('shape inputs X',X.shape)
# output dataset
y = np.array([[0, 0, 1, 1]]).T
print('shape outputs Y',y.shape)
# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)

# initialize weights randomly with mean 0
weights = 2 * np.random.random((3, 1)) - 1


print('weights shape',weights.shape)


: 

Make a loop over 10 000 iteration to update the weights for a ONE LAYER neural network with 1 ouput in the loop you should start by evaluating the output with the current weights then you compute the error made
to update your weights, you can use the 'gradient method' finally you need to update your weights. 



<img src="./figures/nn_simple_1.jpg" width="800" height="400">


In [ ]:
def sig(x):
    return 1/(1+np.exp(-x))

: 

In [ ]:
def dsig(x):
    return np.exp(x)/((np.exp(x)+1)**2)

: 

In [ ]:
weights = 2 * np.random.random((3, 1)) - 1
a0 = X
a1 = y
print(a0)
print(a1.shape)
print(weights.shape)

: 

In [ ]:
weights = 2 * np.random.random((3, 1)) - 1
a0 = X
print('a0 shape',a0.shape)
a1 = y
eta=.5 #learning rate
print(a1.shape)
for iter in range(1000):
    # forward propagation => this is the formula for a_O^(1) in the first image above. 
    a1_hat =sig(np.dot(a0,weights))# rem: this is computing the outputs of all inputs at once! (because a0 is )
    # how much did we miss (a1_hat is our estimation of the result)?- again this is an array containing the errors for all samples
    a1_error = a1-a1_hat
    # compute the gradient of C (multiply how much we missed by the slope of the sigmoid at x* (see slide))
    gradcost =a1_error*sig(a1_hat)
    #print('a1_delta shape',a1_delta.shape)
    # update weights (the dot product is to sum over the whole training set)
    weights +=np.dot(a0.T,gradcost)
    
print('Output After Training:')
print(a1_hat)
print('True output')
print(a1)



: 

# Exercise: Resampling Methods

In [ ]:
myX = np.loadtxt("/Users/fiend/OneDrive/VUB/Master1/Computational Physics/MachineLearning/myX.txt")
myY= np.genfromtxt("/Users/fiend/OneDrive/VUB/Master1/Computational Physics/MachineLearning/myY.txt",delimiter='\n',dtype=str)

: 

: 

In [ ]:
clf = LDA()
clf.fit(myX,myY)

: 

In [ ]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


# preparing cross validation splitting (into 5 folds, 15 times)
rkf = RepeatedKFold(n_splits=5, n_repeats=1)

# getting error values for each fold and for each repetition as (1-the accuracy from clas.score).
# (This is the classification problem and one needs to use the ratio between misclassified cells
# and total number of cells instead of MSE.)
fraction_of_correct_evaluations = cross_val_score(clf, myX, myY, cv=rkf)
print('For the 5 differents splits, the faction of correclty predicted category is')
print(fraction_of_correct_evaluations)
print('For the 5 differents splits, the faction of NOT correclty predicted category is')
print(1-fraction_of_correct_evaluations)
print('Thus, the estimated tMSE is ',np.mean(1-fraction_of_correct_evaluations))


: 

: 